In [479]:
from matplotlib import pylab
import nengo
import random
import numpy as np
import gzip as gz
from cPickle import load
try:
    import Image
except ImportError:
    from PIL import Image
from scipy.sparse.linalg import svds
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#%matplotlib inline #Makes visualizations appar inline (Commeented out because animation popup as new window)

In [480]:
def load_img(path, dims):
    """Load the image at path and return an array representing the raster.
    Flattens image. Shifts pixel activations such that 0 represents gray,
    normalizes the output array.
    Keyword arguments:
    path -- str, path of the image to be loaded.
    dims -- (w, h), where w,h are ints indicating dimensions of the image (in
        px)."""

    img = Image.open(path).resize(dims).getdata()
    img.convert('L')
    img = subtract(array(img).flatten(), 127.5)
    return img/norm(img)


def load_data(filename):
    """Uncompress, unpickle and return a .pkl.gz file.
    Keyword arguments:
    filename -- str, a valid file path"""

    return load(gz.open(filename))

def load_mini_mnist(option=None):
    """Load and return the first \%10 of the images in the mnist dataset.
    Does not return labels. Pass in 'train', 'valid' or 'test' if you want to
    load a specific subset of the dataset.
    Keyword arguments:
    option -- str (default=None)."""

    mini_mnist = load(gz.open('./mini_mnist.pkl.gz', 'rb'))
    if option == 'train':
        return mini_mnist[0]
    elif option == 'valid':
        return mini_mnist[1]
    elif option == 'test':
        return mini_mnist[2]
    else:
        return mini_mnist

In [481]:
def rotate_img(img, degrees):
    '''Rotates image the degrees passed in counterclockwise
    Reshapes image to original shape
    '''
    original = img.shape
    
    newImg = scipy.ndimage.interpolation.rotate(np.reshape(img, (dim,dim), 'F'),degrees,reshape=False)
    newImg = np.reshape(newImg, original, 'F')
    return newImg

In [482]:
conn_synapse = 0.1 #post synaptic time constant to use for filtering (pstc)
probe_synapse = 0.01 #pstc
multiplier = 2 #not used
n_neurons = 5000
direct = False #Direct - function computed explicitly instead of in neurons 
stop_time = 1.0
run_time = 1.2 #in seconds

In [483]:
dim = 28 #size of the image
mnist = load_mini_mnist()
train = mnist[0] #collection of training images
img = mnist[1][0] #image to be used for testing
compress_size = 400 #?
basis, S, V = svds(train.T, k=compress_size) #Used for encoding and decoding information 
    #a set of vectors representing what a hand drawn number should look like?

In [484]:
#Need same number of vectors in basis as number of neurons (randomly sample from basis)
expanded_basis = np.array([random.choice(basis.T) for _ in range(n_neurons)]) 

In [485]:
def stim_func(t):
    '''returns the image for first 0.1s'''
    if t < 0.1:
        return img
    else:
        return [0 for _ in range(len(img))]

In [486]:
def connection_func(x):
    '''takes the output from the first ensemble and rotates it 10 degrees'''
    return rotate_img(x,10)

In [487]:
# Interference from stimulus when trying to rotate, so tried to delay rotation function

def node_func(t,x):
    '''takes the output from the first ensemble and rotates it 10 degrees'''
    if t < 0.1:
        return x
    else:
        return rotate_img(x,10)

In [488]:
with nengo.Network() as net:
    
    if direct:
        neuron_type = nengo.Direct() #function computed explicitly, instead of in neurons
    else:
        neuron_type = nengo.LIF() #spiking version of the leaky integrate-and-fire neuron model

    #Input stimulus - provide data to the ensemble
    ipt = nengo.Node(stim_func)
    
    #Group of neurons that collectively represent information(vector)
    ens = nengo.Ensemble(n_neurons,
                         dimensions=dim**2, #pixels of the image? 28*28
                         encoders=expanded_basis, #transform representational space to neuron space
                         eval_points=expanded_basis, #used for decoder solving, spanning interval
                         n_eval_points=n_neurons,
                         neuron_type=neuron_type)

    nengo.Connection(ipt, #source nengo object
                     ens, #destination object
                     synapse=None, #pstc
                     transform=1) #linear transformation
    
    '''Connection has param solver - (tried to make weight stronger, for image to last longer)
    solver : Solver, optional (Default: nengo.solvers.LstsqL2())
    Solver instance to compute decoders or weights (see Solver). If solver.weights is True, a full 
    connection weight matrix is computed instead of decoders.'''
    #s = nengo.solvers.LstsqL2(weights=True)
    
    conn = nengo.Connection(ens, ens, synapse=conn_synapse, function=connection_func)
    
    #delaynode = nengo.Node(node_func, size_in = dim**2, size_out =dim**2) #does processing on information
    #conn = nengo.Connection(ens, delaynode, synapse=conn_synapse)
    #conn2 = nengo.Connection(delaynode, ens, synapse=conn_synapse)
    
    probe = nengo.Probe(ens, attr='decoded_output',#sample_every=0.001,
                       synapse=probe_synapse)
   

In [489]:
'''
   ens2 = nengo.Ensemble(n_neurons,
                         dimensions=dim**2, #pixels of the image? 28*28
                         encoders=expanded_basis, #transform representational space to neuron space
                         eval_points=expanded_basis, #used for decoder solving, spanning interval
                         n_eval_points=n_neurons,
                         neuron_type=neuron_type)
                         
    nengo.Connection(ipt,ens2,synapse=None,transform=1)
    nengo.Connection(ens,ens2)
    nengo.Connection(ens2,ens)
    nengo.Connection(ens2,ens2,synapse=conn_synapse, function=connection_func)
    
    out = nengo.Node(size_in = dim**2)
    nengo.Connection(ens,out)
    nengo.Connection(ens2,out)
    
    probe = nengo.Probe(ens, attr='decoded_output',synapse=probe_synapse)
'''

"\n   ens2 = nengo.Ensemble(n_neurons,\n                         dimensions=dim**2, #pixels of the image? 28*28\n                         encoders=expanded_basis, #transform representational space to neuron space\n                         eval_points=expanded_basis, #used for decoder solving, spanning interval\n                         n_eval_points=n_neurons,\n                         neuron_type=neuron_type)\n                         \n    nengo.Connection(ipt,ens2,synapse=None,transform=1)\n    nengo.Connection(ens,ens2)\n    nengo.Connection(ens2,ens)\n    nengo.Connection(ens2,ens2,synapse=conn_synapse, function=connection_func)\n    \n    out = nengo.Node(size_in = dim**2)\n    nengo.Connection(ens,out)\n    nengo.Connection(ens2,out)\n    \n    probe = nengo.Probe(ens, attr='decoded_output',synapse=probe_synapse)\n"

In [490]:
sim = nengo.Simulator(net)

In [491]:
sim.run(run_time)

Simulation finished in 0:00:10.                                                 


In [492]:
pylab.imshow(np.reshape(img, (dim,dim), 'F'), cmap='Greys_r')

In [493]:
'''Image at stop time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][int(stop_time*1000)]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [496]:
'''Animation for Probe output'''
fig = plt.figure()

def updatefig(i):
    im = pylab.imshow(np.reshape([0. if x < 0.00001 else x 

                              for x in sim.data[probe][i]], (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),
                  animated=True)
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=1, blit=True)
plt.show()
